# NEWEST 

In [104]:
# IW CODE: https://github.com/sarahmbock/COS-IW-BOCK/blob/main/DANN%20CODE.ipynb

# This file uses the code from DANN_Adult, not unweight
# It uses the farcon data
# it includes parity calculations

In [20]:
from __future__ import print_function
import argparse
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms
import numpy as np
import torch.utils.data as utils
import time
import os
import torch.nn as nn
from torch.autograd import Function
import torchvision
import torch.utils.data as data
from PIL import Image
import os
from torch.utils.data import TensorDataset, DataLoader
import torch.optim as optim
from torch.utils.data import DataLoader, WeightedRandomSampler
import numpy as np 
import torch.nn.functional as F
import torch.nn.init as init
import pandas as pd
import sklearn.preprocessing as preprocessing

In [21]:
# Gradient reversal layer
class GRL(Function):
    @staticmethod
    def forward(ctx, x, constant):
        ctx.constant = constant
        return x.view_as(x) * constant
    @staticmethod
    def backward(ctx, grad_output):
        return grad_output.neg() * ctx.constant, None

In [24]:
import torch.nn.functional as F
from torch.autograd import Function

class GradReverse(Function):
    
    @staticmethod
    def forward(ctx, x, lambda_):
        ctx.lambda_ = lambda_
        return x.view_as(x)
    
    @staticmethod
    def backward(ctx, grad_output):
        output = grad_output.neg() * ctx.lambda_
        return output, None

def grad_reverse(x, lambda_=1.0):
    return GradReverse.apply(x, lambda_)


In [25]:
# Can adjust NN Sizes
class Dann(nn.Module):
    def __init__(self):
        super(Dann, self).__init__()
        self.f = nn.Sequential(
                nn.Linear(95, 64),
            nn.ReLU(),
        )
        # Label classifier
        self.lc = nn.Sequential(
    nn.Linear(64, 36), # changed 32 to 36
    nn.Sigmoid(),
    nn.Linear(36, 32),
    nn.Sigmoid(),
    nn.Linear(32, 2),
    nn.Sigmoid()
)
        
        # Domain classifier
        self.dc = nn.Sequential(nn.Linear(64, 38),
    nn.Sigmoid(),
    nn.Linear(38, 38), nn.Sigmoid(),              
    nn.Linear(38, 2),
    nn.Sigmoid()
                                
)
    def forward(self, x,alpha):
        x = self.f(x)
        x = x.view(-1, 64) # must be same dimension as output size from init()
        y = GRL.apply(x, alpha)
        x = self.lc(x)
        y = self.dc(y)
        x=x.view(x.shape[0],-1)
        y=y.view(y.shape[0],-1)
        return x, y

# DATA

In [31]:
# # Adult data code
# # Change these to local file if available
# train_url = 'https://archive.ics.uci.edu/ml/machine-learning-databases/adult/adult.data'
# test_url = 'https://archive.ics.uci.edu/ml/machine-learning-databases/adult/adult.test'

# features = ["Age", "Workclass", "fnlwgt", "Education", "Education-Num", "Martial Status",
#         "Occupation", "Relationship", "Race", "Sex", "Capital Gain", "Capital Loss",
#         "Hours per week", "Country", "Target"] 


# # This will download 3.8M
# original_train = pd.read_csv(train_url, names=features, sep=r'\s*,\s*', 
#                              engine='python', na_values="?")
# # This will download 1.9M
# original_test = pd.read_csv(test_url, names=features, sep=r'\s*,\s*', 
#                             engine='python', na_values="?", skiprows=1)



# num_train = len(original_train)
# original = pd.concat([original_train, original_test])
# roc_original = original
# labels = original['Target']
# labels = labels.replace('<=50K', 0).replace('>50K', 1)
# labels = labels.replace('<=50K.', 0).replace('>50K.', 1)


# # Preprocess
# # Redundant column
# del original["Education"]
# # Remove target variable
# del original["Target"]


# def data_transform(df):
#     """Normalize features."""
#     binary_data = pd.get_dummies(df)
#     feature_cols = binary_data[binary_data.columns[:-2]]
#     scaler = preprocessing.StandardScaler()
#     data = pd.DataFrame(scaler.fit_transform(feature_cols), columns=feature_cols.columns)
#     return data

# data = data_transform(original)

In [26]:
test_data = pd.read_csv('adult_test_bin.csv')
train_data = pd.read_csv('adult_train_bin.csv')
complete_data = pd.concat([train_data, test_data])

sensitive_col = 'gender_ Male'
# switch 0 and 1 for sensitive attribute so female is 1
complete_data[sensitive_col] = complete_data[sensitive_col].map({0: 1, 1: 0})
target_col = 'income_ >50K'
labels = complete_data[target_col]
data = complete_data

In [27]:
# Adult code, seperate sensitive and non sensitive
# get array of sensitive characteristics
sensitive = complete_data[sensitive_col]
y = np.array(labels).flatten()
print(data.columns)
data = data.drop([target_col, sensitive_col], axis=1) # Drop sensitive info from data
data_np = np.array(data)

Index(['gender_ Male', 'age', 'fnlwgt', 'education-num', 'capital-gain',
       'capital-loss', 'hours-per-week', 'workclass_ Local-gov',
       'workclass_ Private', 'workclass_ Self-emp-inc',
       'workclass_ Self-emp-not-inc', 'workclass_ State-gov',
       'workclass_ Without-pay', 'education_ 11th', 'education_ 12th',
       'education_ 1st-4th', 'education_ 5th-6th', 'education_ 7th-8th',
       'education_ 9th', 'education_ Assoc-acdm', 'education_ Assoc-voc',
       'education_ Bachelors', 'education_ Doctorate', 'education_ HS-grad',
       'education_ Masters', 'education_ Preschool', 'education_ Prof-school',
       'education_ Some-college', 'martial-status_ Married-AF-spouse',
       'martial-status_ Married-civ-spouse',
       'martial-status_ Married-spouse-absent',
       'martial-status_ Never-married', 'martial-status_ Separated',
       'martial-status_ Widowed', 'occupation_ Armed-Forces',
       'occupation_ Craft-repair', 'occupation_ Exec-managerial',
       'o

In [28]:
# split to source (non-sensitive) and target (sensitive)
sensitive_idx = np.array(np.where(sensitive==1))[0].flatten()
nonsensitive_idx = np.array(np.where(sensitive!=1))[0].flatten()
data_sensitive = data_np[sensitive_idx,:]
data_nonsensitive = data_np[nonsensitive_idx,:]
y_sensitive = y[sensitive_idx]
y_nonsensitive = y[nonsensitive_idx]





In [8]:
# # ORIGINAL CODE -- need to replace with adult code


# import pandas as pd
# import numpy as np
# #import rowgenerators as rg ****commented out for thesis****

# from sklearn.feature_selection import VarianceThreshold

# scfp = pd.read_stata('p19i6.dta')



# # SELECT only rows where there is debt and where white and black
# delinquency = 'x3005'
# loan_schedule ='x3004'# 1 on time, 5 sometimes late, 0 inapp.
# race = 'x6809'
# additional_race = 'x6810'
# hispanic = 'x7004'

# scfp = scfp[(scfp[race] == 1) | (scfp[race] == 2)]
# scfp =  scfp[(scfp[loan_schedule] == 1) | (scfp[loan_schedule] == 5)]
# scfp.shape

# # Remove revealing columns
# race_revealing = [additional_race, 'x7004']
# race_revealing += [col.replace('x', 'j') for col in race_revealing]
# sched_revealing = ['x7571', 'x7570', 'x7569', 'x7556', 'x7564', 'x7554','x7553',
#                    'x7534', 'x7533', 'x7532', 'x7166', 'x7529', 'x7821', 'x7844',
#                    'x7867', 'x7921', 'x7569', 'x7944', 'x7521', 'x7520', 'x7518',
#                    'x7517', delinquency, 'j3005', 'j3004']
# sched_revealing += [col.replace('x', 'j') for col in sched_revealing]

# exclude = sched_revealing+ race_revealing
# X = scfp.copy()
# # Eliminate bad columns
# X = scfp.loc[:, [col for col in scfp.columns if col not in exclude]]
# X = X.fillna(-1)

# # Remove columns with all -1

# # instantiate the VarianceThreshold object with a threshold of 0
# selector = VarianceThreshold(threshold=0)

# # fit the selector on the DataFrame
# selector.fit(X)

# # get the indices of the non-constant columns
# non_constant_cols = selector.get_support(indices=True)

# # select the non-constant columns from the original DataFrame
# X = X.iloc[:, non_constant_cols]

# # print the filtered DataFrame
# print(X.shape)

In [113]:
# from sklearn.model_selection import train_test_split
# from torch.utils.data import TensorDataset, DataLoader

# # split to source (white) and target (black)
# # First, create a new column in X that indicates whether each person is white or black
# print(len(X))
# # Use groupby to split X and y into two separate dataframes, one with only white data and the other with only black data
# X_white = X[X[race] == 1]
# y_white = X_white[loan_schedule].replace({1: 0, 5: 1})
# X_white = X_white.drop([race, loan_schedule], axis =1)
# #print(y_white)
# print(len(y_white))
# print(len(X_white))

# X_black = X[X[race]==2]
# y_black = X_black[loan_schedule].replace({1: 0, 5: 1})
# X_black = X_black.drop([race, loan_schedule], axis =1)
# #print(y_black)
# print(len(y_black))
# print(len(X_black))

In [29]:
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from torch.utils.data import TensorDataset, DataLoader

X_tensor_source =  torch.from_numpy(data_nonsensitive).type(torch.float32)
X_tensor_target =  torch.from_numpy(data_sensitive).type(torch.float32)
y_tensor_source =torch.tensor(y_nonsensitive).type(torch.LongTensor)
y_tensor_target =torch.tensor(y_sensitive).type(torch.LongTensor)


# Split the source (white) dataset into training and test sets
X_train_source, X_test_source, y_train_source, y_test_source = train_test_split(X_tensor_source, y_tensor_source, test_size=0.2, random_state=42)


# Split the target (black) into training and test sets
X_train_target, X_test_target, y_train_target, y_test_target = train_test_split(X_tensor_target, y_tensor_target, test_size=0.2, random_state=42)

# Create TensorDatasets from the training and test sets
target_train = TensorDataset(X_train_target, y_train_target)
target_test = TensorDataset(X_test_target, y_test_target)

source_train = TensorDataset(X_train_source, y_train_source)
source_test = TensorDataset(X_test_source, y_test_source)

# WEIGHTS
num_samples = len(target_train) # number of black samples
num_positives = sum(target_train[i][1] for i in range(num_samples)) # Number black with positive label
num_negatives = num_samples - num_positives
class_sample_count = np.array([num_negatives, num_positives]) 


weight = 1/class_sample_count
print('positive weight (under represented):', weight[1])
print('negative weight (over represented):', weight[0])

samples_weight = np.array([weight[t] for t in y_train_target])
samples_weight = torch.from_numpy(samples_weight)
sampler = WeightedRandomSampler(samples_weight.type('torch.DoubleTensor'), len(samples_weight))




# Create DataLoaders from the datasets
batch_size = 64
target_train = DataLoader(target_train, batch_size=batch_size, sampler=sampler) # remove sampler
target_test= DataLoader(target_test, batch_size=len(target_test), shuffle=False)



num_samples = len(source_train)
num_positives = sum(source_train[i][1] for i in range(num_samples))
num_negatives = num_samples - num_positives
class_sample_count = np.array([num_negatives, num_positives])

weight = 1/class_sample_count
print('positive weight (under represented):', weight[1])
print('negative weight (over represented):', weight[0])

samples_weight = np.array([weight[t] for t in y_train_source])
samples_weight = torch.from_numpy(samples_weight)
sampler = WeightedRandomSampler(samples_weight.type('torch.DoubleTensor'), len(samples_weight))


source_train = DataLoader(source_train, batch_size=batch_size, sampler=sampler) # remove sampler
source_test = DataLoader(source_test, batch_size=len(source_test), shuffle=False)

print(X_train_source.shape)
print(X_train_target.shape)

positive weight (under represented): 0.0007513148009015778
negative weight (over represented): 9.592326139088728e-05
positive weight (under represented): 0.00013243279035889287
negative weight (over represented): 5.927682276229994e-05
torch.Size([24421, 95])
torch.Size([11756, 95])


In [30]:
combined_tensor = torch.cat((X_train_source, X_train_target), dim=0)
# Weight with respect to sensitive feature itself
y_domain = torch.zeros(len(combined_tensor), dtype=torch.long)
y_domain[len(X_train_source):] = 1
print(sum(y_domain).item())
print(y_domain)
combined_set =  TensorDataset(combined_tensor, y_domain)
class_sample_count = np.array([len(X_train_source), len(X_train_target)])
print(class_sample_count)
weight = 1/class_sample_count
print(weight)
samples_weight = np.array([weight[t] for t in y_domain])
samples_weight = torch.from_numpy(samples_weight)
sampler = WeightedRandomSampler(samples_weight.type('torch.DoubleTensor'), len(samples_weight))

combined_loader =  DataLoader(combined_set, batch_size=batch_size) # Removed sensitive characteristic data loader



# Combine white and black to train credit predictor
y_label_combined = torch.cat((y_train_source,y_train_target), dim=0)
X_label_combined = torch.cat((X_train_source,X_train_target), dim=0)
num_samples = len(X_label_combined)
num_positives = sum(y_label_combined[i] for i in range(num_samples)) # Number of positives for label predictor
num_negatives = num_samples - num_positives
print(num_positives.item())
print(num_negatives.item())
class_sample_count = np.array([num_negatives.item(), num_positives.item()])
weight = 1/class_sample_count
print(weight)
samples_weight = np.array([weight[t] for t in y_label_combined])
samples_weight = torch.from_numpy(samples_weight)
sampler = WeightedRandomSampler(samples_weight.type('torch.DoubleTensor'), len(samples_weight))
combined_label_set = TensorDataset(X_label_combined, y_label_combined)
combined_label_loader = DataLoader(combined_label_set, batch_size=batch_size) # Remove sampler

11756
tensor([0, 0, 0,  ..., 1, 1, 1])
[24421 11756]
[4.09483641e-05 8.50629466e-05]
8882
27295
[3.66367467e-05 1.12587255e-04]


In [31]:
def test(model, source_train_loader, source_test_loader, target_test_loader, lambda_):
    # Set model to evaluation mode
    model.eval()
    model.lc.eval()
    model.dc.eval()
    model.f.eval()

    # Define variables for tracking accuracy
    source_label_correct = 0
    source_domain_correct = 0
    target_domain_correct = 0

    # Evaluate on source train set
    for i, (data, labels) in enumerate(source_train_loader):
        # Set domain label to 0 (i.e., source domain)
        domain_labels = torch.zeros(data.shape[0]).long()

        # Forward pass through the model
        label_preds, domain_preds = model(data, alpha=lambda_)
        #print(domain_preds)
        domain_preds = model.dc(grad_reverse(model.f(data), lambda_))
        

        # Calculate accuracy of label classifier
        _, label_preds = torch.max(label_preds, 1)
        source_label_correct += (label_preds == labels).sum().item()

        # Calculate accuracy of domain classifier
        _, domain_preds = torch.max(domain_preds, 1)
        source_domain_correct += (domain_preds == domain_labels).sum().item()
    #num_ones_predicted =0
    # Evaluate on source test set
    for i, (data, labels) in enumerate(source_test_loader):
        # Set domain label to 0 (i.e., source domain)
        domain_labels = torch.zeros(data.shape[0]).long()

        # Forward pass through the model
        label_preds, domain_preds = model(data, alpha=lambda_)
        domain_preds = model.dc(grad_reverse(model.f(data), lambda_))

        # Calculate accuracy of label classifier
        _, label_preds = torch.max(label_preds, 1)
        source_label_correct += (label_preds == labels).sum().item()
        #num_ones_predicted += list(label_preds.numpy()).count(1)

        # Calculate accuracy of domain classifier
        _, domain_preds = torch.max(domain_preds, 1)
        source_domain_correct += (domain_preds == domain_labels).sum().item()
    
    # Evaluate on target test set
    target_label_correct =0
    for i, (data, labels) in enumerate(target_test_loader):
        # Set domain label to 1 (i.e., target domain)
        domain_labels = torch.ones(data.shape[0]).long()

        # Forward pass through the model
        label_preds, domain_preds = model(data, alpha=lambda_)
        domain_preds = model.dc(grad_reverse(model.f(data), lambda_))
        
        # Calculate accuracy of label classifier
        _, label_preds = torch.max(label_preds, 1)
        target_label_correct += (label_preds == labels).sum().item()
        #num_ones_predicted += list(label_preds.numpy()).count(1)
        
        # Calculate accuracy of domain classifier
        _, domain_preds = torch.max(domain_preds, 1)
        target_domain_correct += (domain_preds == domain_labels).sum().item()
        

    # Calculate and print accuracy
    source_label_accuracy = 100. * source_label_correct / (len(source_train_loader.dataset) + len(source_test_loader.dataset))
    source_domain_accuracy = 100. * source_domain_correct / (len(source_train_loader.dataset) + len(source_test_loader.dataset))
    target_label_accuracy = 100. * target_label_correct / (len(target_test_loader.dataset))
    target_domain_accuracy = 100. * target_domain_correct / len(target_test_loader.dataset)
    print('Source Label Accuracy: {:.2f}%'.format(source_label_accuracy))
    print('Target Label Accuracy: {:.2f}%'.format(target_label_accuracy))
    print('Source Domain Accuracy: {:.2f}%'.format(source_domain_accuracy))
    print('Target Domain Accuracy: {:.2f}%'.format(target_domain_accuracy))
    #print('Number of loan defaults predicted: ', num_ones_predicted / (len(target_test_loader.dataset)+ len(source_test_loader.dataset)))


In [32]:
from sklearn.metrics import confusion_matrix
from fairlearn.metrics import MetricFrame
from sklearn.metrics import accuracy_score
def test_fairness(model, lambda_):
    with torch.no_grad():
        predicted_labels_white = []
        true_labels_white = []
        predicted_domain = []
        source_label_correct =0
        source_domain_correct =0
        for i, (data, labels) in enumerate(source_test):
            # Set domain label to 0 (i.e., source domain)
            domain_labels = torch.zeros(data.shape[0]).long()

            # Forward pass through the model
            label_preds, domain_preds = model(data, alpha=lambda_)
            domain_preds = model.dc(grad_reverse(model.f(data), lambda_=lambda_))

            # Calculate accuracy of label classifier
            _, label_preds = torch.max(label_preds, 1)
            source_label_correct += (label_preds == labels).sum().item()
            #num_ones_predicted += list(label_preds.numpy()).count(1)

            # Calculate accuracy of domain classifier
            _, domain_preds = torch.max(domain_preds, 1)
            source_domain_correct += (domain_preds == domain_labels).sum().item()
            predicted_labels_white += label_preds
            true_labels_white += labels
            predicted_domain += domain_preds
        conf_matrix = confusion_matrix(true_labels_white, predicted_labels_white)
        print('Credit Confusion Matrix White: ')
        print(conf_matrix)
        fpr = conf_matrix[0, 1] / (conf_matrix[0, 1] + conf_matrix[0, 0]) 
        print('False positive white: ', fpr)
        
        predicted_labels = []
        true_labels = []
        predicted_domain = []
        target_label_correct =0
        target_domain_correct =0
        for i, (data, labels) in enumerate(target_test):
            # Set domain label to 0 (i.e., source domain)
            domain_labels = torch.ones(data.shape[0]).long()

            # Forward pass through the model
            label_preds, domain_preds = model(data, alpha=lambda_)
            domain_preds = model.dc(grad_reverse(model.f(data), lambda_=lambda_))

            # Calculate accuracy of label classifier
            _, label_preds = torch.max(label_preds, 1)
            target_label_correct += (label_preds == labels).sum().item()
            #num_ones_predicted += list(label_preds.numpy()).count(1)

            # Calculate accuracy of domain classifier
            _, domain_preds = torch.max(domain_preds, 1)
            target_domain_correct += (domain_preds == domain_labels).sum().item()
            predicted_labels += label_preds
            true_labels += labels
            predicted_domain += domain_preds
        conf_matrix = confusion_matrix(true_labels, predicted_labels)
        print('Credit Confusion Matrix Black: ')
        print(conf_matrix)
        fpr = conf_matrix[0, 1] / (conf_matrix[0, 1] + conf_matrix[0, 0]) 
        print('False positive black: ', fpr)
        
        # predictive Parity
        metric_frame = MetricFrame(metrics=accuracy_score, y_true=true_labels_white + true_labels,
                           y_pred=predicted_labels_white + predicted_labels,
                           sensitive_features=[1]*len(true_labels_white) + [0]*len(true_labels)) #are white and black mixed up here?

        predictive_parity = metric_frame.difference(method='between_groups')

        print("Predictive parity: ", predictive_parity)
        
        
        
    

In [33]:
def test_lc(model, lambda_):
    model.eval()
    model.lc.eval()
    model.f.eval()
    correct = 0
    ones = 0
    for data, labels in source_test:
        embedding = model.f(data)
        label_preds = model.lc(embedding)
        print(label_preds)
        label_preds = torch.max(label_preds, 1)
        print(label_preds)
        # Get number of correct predictions
        source_label_correct += (label_preds == labels).sum().item()
        ones += list(label_preds.numpy()).count(1)
        
        # get number of ones
    source_test_label_acc = source_label_correct / len(source_test_loader.dataset)
    print('Accuracy of label classifier on source test: ', source_test_label_acc )
    print('Late loans predicted on source test: ', ones/ len(source_test_loader.dataset))
        
        

In [34]:
def test_dc(model, lambda_):
    model.eval()
    model.lc.eval()
    model.f.eval()
    model.dc.eval()
    correct = 0
    black = 0
    actually_black = 0
    for data, labels in combined_loader:
        combined_data, domain_labels = data, labels
        embedded = grad_reverse(model.f(combined_data), lambda_=lambda_)
        domain_preds = model.dc(embedded)
        #print(domain_preds)
        _, domain_preds = torch.max(domain_preds, 1)
        #domain_loss = domain_criterion(domain_preds, domain_labels)
        #print(domain_preds)
        correct += (domain_preds == labels).sum().item()
        black += list(domain_preds.numpy()).count(1)
        actually_black += list(labels.numpy()).count(1)
    accuracy = correct / len(combined_loader.dataset)
    print('domain accuracy on train data: ', accuracy)
    print('black predicted: ', black/(len(combined_loader.dataset)))
    print('num actually black: ', actually_black/(len(combined_loader.dataset)))
    black = 0
    correct = 0
    actually_black = 0
    for data, labels in source_test:
        embedded = grad_reverse(model.f(data), lambda_=lambda_)
        domain_preds = model.dc(embedded)
        _, domain_preds = torch.max(domain_preds, 1)
        labels = torch.zeros(data.shape[0]).long()
        correct += (domain_preds == labels).sum().item()
        black += list(domain_preds.numpy()).count(1)
    for data, labels in target_test:
        embedded = grad_reverse(model.f(data), lambda_=lambda_)
        domain_preds = model.dc(embedded)
        _, domain_preds = torch.max(domain_preds, 1)
        labels = torch.ones(data.shape[0]).long()
        correct += (domain_preds == labels).sum().item()
        black += list(domain_preds.numpy()).count(1)
    test_accuracy = correct / (len(target_test.dataset)+ len(source_test.dataset))
    print('domain test accuracy: ', test_accuracy)
    print('black pred test: ', black/(len(target_test.dataset)+ len(source_test.dataset)))
    print('actual black = ', len(target_test.dataset)/(len(target_test.dataset)+ len(source_test.dataset)))
          
        
        

        
    
    
    

In [36]:
def train(dann, num_epochs, feature_optimizer, label_epochs, domain_epochs, source_loader, target_loader, label_criterion, domain_criterion,
         label_optimizer, domain_optimizer, lambda_):
    for epoch in range(num_epochs):
    # Set the DANN model to training mode
        dann.train()
        # Set the number of batches for each epoch

            
        # Train the label classifier for label_epochs epochs
        for j in range(label_epochs):
            dann.lc.train()
            for data, labels in combined_label_loader:
                source_data, source_labels = data, labels
                label_optimizer.zero_grad()
                feature_embed = dann.f(source_data)
                label_preds = dann.lc(feature_embed)
                label_loss = label_criterion(label_preds, source_labels)
                label_loss.backward()
                label_optimizer.step()
        print('label trained')
        dann.lc.eval()

        # Train the domain classifier for domain_epochs epochs
        for j in range(domain_epochs):
            dann.dc.train()
            for data, labels in combined_loader:
                combined_data, domain_labels = data, labels
                domain_optimizer.zero_grad()
                embedded = grad_reverse(dann.f(combined_data), lambda_=lambda_)
                domain_preds = dann.dc(embedded)
                domain_loss = domain_criterion(domain_preds, domain_labels)
                domain_loss.backward()
                domain_optimizer.step()
        print('domain trained')
        dann.dc.eval()


        dann.f.train()
        dann.dc.train()
        dann.lc.train()
        dann.train()
        
        for data, labels in source_loader:
            feature_optimizer.zero_grad()
            #dann.zero_grad()
            #feature_embed_label, feature_embed_domain = dann(data, lambda_)
            #feature_embed_domain = grad_reverse(dann.f(data), lambda_=lambda_)
            #label_preds = dann.lc(feature_embed_label)
            #domain_preds = dann.dc(feature_embed_domain)
            label_preds, domain_preds = dann(data, lambda_)
            label_loss = label_criterion(label_preds, labels)
            domain_labels = torch.zeros(len(data), dtype=torch.long)
            domain_loss = domain_criterion(domain_preds, domain_labels)
            total_loss = label_loss + (domain_loss* lambda_)
            total_loss.backward()
            feature_optimizer.step()
            
        for data, labels in target_loader:
            feature_optimizer.zero_grad()
            #dann.zero_grad()
            #feature_embed = dann.f(data)
            #label_preds = dann.lc(feature_embed)
            #feature_embed_domain = grad_reverse(dann.f(data), lambda_=lambda_)
            #domain_preds=dann.dc(feature_embed_domain)
            #print(domain_preds)
            label_preds, domain_preds = dann(data, lambda_)
            label_loss = label_criterion(label_preds, labels)
            domain_labels = torch.ones(len(data), dtype=torch.long)
            domain_loss = domain_criterion(domain_preds, domain_labels)
            total_loss = label_loss + (domain_loss*lambda_)
            total_loss.backward()
            feature_optimizer.step()
        print('feature extractor trained')
        dann.f.eval()
        dann.dc.eval()
        dann.lc.eval()
        dann.eval()
        print('Epoch [{}/{}], Label Loss: {:.4f}, Domain Loss: {:.4f}'.format(epoch+1, num_epochs, label_loss.item(), domain_loss.item()))
        test(dann, source_train, source_test, target_test, lambda_)
        #test_dc(dann, lambda_)
        #test_fairness(dann, lambda_)
        print()

# Print the loss for this epoch
    
    


In [37]:
def train_for_lambda(lambda_):
    model = Dann()

    # initialize weights
    for m in model.f.modules():
        if isinstance(m, nn.Linear):
            init.xavier_uniform_(m.weight)
    for m in model.lc.modules():
        if isinstance(m, nn.Linear):
            init.xavier_uniform_(m.weight)
    for m in model.dc.modules():
        if isinstance(m, nn.Linear):
            init.xavier_uniform_(m.weight)

    # Define optimizer for the feature extractor and label classifier
    optimizer_lc = torch.optim.Adam(list(model.f.parameters()) + list(model.lc.parameters()), lr=0.00001, weight_decay=1e-3)
    criterion_lc = nn.CrossEntropyLoss()

    # Define optimizer for the domain classifier
    optimizer_dc = torch.optim.Adam(model.dc.parameters(), lr=0.0001, weight_decay=1e-3)
    criterion_dc = nn.CrossEntropyLoss()

    f_optimizer = torch.optim.Adam(model.f.parameters(), lr=0.0001, weight_decay=1e-3)
    # Define number of epochs to train the label and domain classifiers
    n_lc_epochs = 10
    n_dc_epochs = 10

    _lambda = lambda_
    lambda_ = _lambda
    # Training loop
    num_epochs = 20
    train(model, num_epochs, optimizer_lc, 30, domain_epochs =30, 
          source_loader=source_train, target_loader= target_train, 
          label_criterion =criterion_lc, domain_criterion=criterion_dc,
         label_optimizer = optimizer_lc, domain_optimizer = optimizer_dc, lambda_ = lambda_)
    return model



In [122]:
'''lambda = 0
label trained
domain trained
feature extractor trained
Epoch [30/30], Label Loss: 0.3613, Domain Loss: 0.3133
Source Label Accuracy: 96.33%
Target Label Accuracy: 97.27%
Source Domain Accuracy: 89.32%
Target Domain Accuracy: 84.73%
domain accuracy on train data:  0.901286677105317
black predicted:  0.4585063653073729
num actually black:  0.48519300156579753
domain test accuracy:  0.906071331336782
black pred test:  0.19793084671930303
actual black =  0.14974135583991288
Credit Confusion Matrix White: 
[[2673   64]
 [  40  346]]
False positive white:  0.023383266350018268
Credit Confusion Matrix Black: 
[[376   7]
 [  8 159]]
False positive black:  0.018276762402088774

'''

'lambda = 0\nlabel trained\ndomain trained\nfeature extractor trained\nEpoch [30/30], Label Loss: 0.3613, Domain Loss: 0.3133\nSource Label Accuracy: 96.33%\nTarget Label Accuracy: 97.27%\nSource Domain Accuracy: 89.32%\nTarget Domain Accuracy: 84.73%\ndomain accuracy on train data:  0.901286677105317\nblack predicted:  0.4585063653073729\nnum actually black:  0.48519300156579753\ndomain test accuracy:  0.906071331336782\nblack pred test:  0.19793084671930303\nactual black =  0.14974135583991288\nCredit Confusion Matrix White: \n[[2673   64]\n [  40  346]]\nFalse positive white:  0.023383266350018268\nCredit Confusion Matrix Black: \n[[376   7]\n [  8 159]]\nFalse positive black:  0.018276762402088774\n\n'

In [38]:
def write_results(lam, model, test_src, test_targ):
    file_name = 'results_mult_adult'+str(lam)+'.txt'
    my_str = 'Lambda: ' +str(lam) + '\n\n'
    results = get_results(model, test_src, test_targ, lam)
    # test accuracy-- source, target, and overall
    my_str += 'Label Classifier Accuracy (Test Data) \n'
    my_str += 'Source (White): ' + str(results['source_label_accuracy']) + '\n'
    my_str += 'Target (Black): ' + str(results['target_label_accuracy'])+ '\n'
    my_str += 'Overall: ' + str(results['overall_label_accuracy'])+ '\n\n'
    # domain test accuracy-- source, target, and overall
    my_str += 'Domain Classifier Accuracy (Test Data) \n'
    my_str += 'Source (White): ' + str(results['source_domain_accuracy']) + '\n'
    my_str += 'Target (Black): ' + str(results['target_domain_accuracy'])+ '\n'
    my_str += 'Overall: ' + str(results['overall_domain_accuracy'])+ '\n\n'
    # false positives-- source, target, and overall
    my_str += 'False Positive Rate (Test Data) \n'
    my_str += 'Source (White): ' + str(results['fpr_white']) + '\n'
    my_str += 'Target (Black): ' + str(results['fpr_black'])+ '\n'
    my_str += '(Black FPR) / (White FPR): ' +  str(results['fpr_black']/results['fpr_white'])+ '\n'
    my_str += 'Overall: ' + str(results['fpr_overall'])+ '\n\n'
    
    print(my_str)
    with open(file_name, "w") as file:
        file.write(my_str)
    
    # parity
    

In [39]:
def get_results(model, test_src, test_targ, lam):
    #print('get results model')
    #print(model)
    with torch.no_grad():
        # Label predictor metrics
        source_correct = 0
        source_label_predictions = []
        source_label_true = []
        
        # Domain predictor metrics
        source_dom_correct = 0
        source_dom_predictions = []
        source_dom_true = []
        for data, labels in test_src:
            # Make label predictions
            embedded = model.f(data)
            label_preds = model.lc(embedded)
            _, label_preds = torch.max(label_preds, 1)
            source_correct += (label_preds == labels).sum().item()
            source_label_predictions += label_preds
            source_label_true+= labels
            
            # Make domain predictions
            domain_embedded = grad_reverse(model.f(data), lam)
            domain_preds = model.dc(domain_embedded)
            _, domain_preds = torch.max(domain_preds, 1)
            domain_true = torch.zeros(data.shape[0]).long()
            source_dom_correct += (domain_preds == domain_true).sum().item()
            source_dom_predictions += domain_preds
            source_dom_true += domain_true
        source_label_accuracy = 100.* source_correct / len(test_src.dataset)
        source_domain_accuracy = 100 * source_dom_correct / len(test_src.dataset)
        source_label_positive_rate = (sum(source_label_predictions)/len(source_label_predictions)).item()
        
        
        
        # FOR TARGET DATA
        # Label predictor metrics
        target_correct = 0
        target_label_predictions = []
        target_label_true = []
        
        # Domain predictor metrics
        target_dom_correct = 0
        target_dom_predictions = []
        target_dom_true = []
        for data, labels in test_targ:
            # Make label predictions
            embedded = model.f(data)
            label_preds = model.lc(embedded)
            _, label_preds = torch.max(label_preds, 1)
            target_correct += (label_preds == labels).sum().item()
            target_label_predictions += label_preds
            target_label_true+= labels
            
            # Make domain predictions
            domain_embedded = grad_reverse(model.f(data), lam)
            domain_preds = model.dc(domain_embedded)
            _, domain_preds = torch.max(domain_preds, 1)
            domain_true = torch.ones(data.shape[0]).long()
            target_dom_correct += (domain_preds == domain_true).sum().item()
            target_dom_predictions += domain_preds
            target_dom_true += domain_true
        target_label_accuracy = 100.* target_correct / len(test_targ.dataset)
        target_domain_accuracy = 100 * target_dom_correct / len(test_targ.dataset)
        target_label_positive_rate = (sum(target_label_predictions)/len(target_label_predictions)).item()
        
        # OVERALL
        overall_label_accuracy = 100.*(target_correct + source_correct)/(len(test_src.dataset)+len(test_targ.dataset))
        overall_domain_accuracy = 100.*(target_dom_correct + source_dom_correct)/(len(test_src.dataset)+len(test_targ.dataset))
        overall_label_positive_rate = ((sum(source_label_predictions) + sum(target_label_predictions))/(len(target_label_predictions)+len(source_label_predictions))).item()
        # False positive rates
        conf_matrix = confusion_matrix(source_label_true, source_label_predictions)
        fpr_white = 100.* conf_matrix[0, 1] / (conf_matrix[0, 1] + conf_matrix[0, 0])
        
        conf_matrix = confusion_matrix(target_label_true, target_label_predictions)
        fpr_black = 100.* conf_matrix[0, 1] / (conf_matrix[0, 1] + conf_matrix[0, 0])
        
        conf_matrix = confusion_matrix(target_label_true+source_label_true, target_label_predictions+source_label_predictions)
        fpr_overall = 100. *conf_matrix[0, 1] / (conf_matrix[0, 1] + conf_matrix[0, 0])
        
        
        
        
        
        
        
        
        results = {'source_label_accuracy': source_label_accuracy, 'source_domain_accuracy':source_domain_accuracy,
                  'target_label_accuracy': target_label_accuracy, 'target_domain_accuracy':target_domain_accuracy,
                   'overall_label_accuracy': overall_label_accuracy, 'overall_domain_accuracy': overall_domain_accuracy,
                   'fpr_white': fpr_white, 'fpr_black': fpr_black, 'fpr_overall': fpr_overall,
                   'source_label_positive_rate': source_label_positive_rate, 'target_label_positive_rate': target_label_positive_rate,
                   'overall_label_positive_rate': overall_label_positive_rate,
                   }
        return results
        
        
            
            
            
                
            
            
    

In [ ]:
# # For adult only, fine tune the model to be more accurate
# lam = 0
# model = train_for_lambda(lam)
# write_results(lam, model, source_test, target_test)



In [ ]:
# lam = 2
# model = train_for_lambda(lam)
# write_results(lam, model, source_test, target_test)

In [125]:
#write_results(0.8, model, source_test, target_test)

In [19]:
models = {}
# Changed hidden size of dc 36 to 38
for lam in range(10):
    lam = 0.1*lam
    model = train_for_lambda(lam)
    models[lam] = model
    write_results(lam, model, source_test, target_test)
    
    
    

label trained
domain trained
feature extractor trained
Epoch [1/20], Label Loss: 0.7761, Domain Loss: 0.9923
Source Label Accuracy: 53.37%
Target Label Accuracy: 88.50%
Source Domain Accuracy: 100.00%
Target Domain Accuracy: 0.00%

label trained
domain trained
feature extractor trained
Epoch [2/20], Label Loss: 0.5878, Domain Loss: 1.0064
Source Label Accuracy: 75.19%
Target Label Accuracy: 91.60%
Source Domain Accuracy: 83.85%
Target Domain Accuracy: 37.60%

label trained
domain trained
feature extractor trained
Epoch [3/20], Label Loss: 0.4844, Domain Loss: 1.0186
Source Label Accuracy: 77.15%
Target Label Accuracy: 90.00%
Source Domain Accuracy: 82.31%
Target Domain Accuracy: 42.06%

label trained
domain trained
feature extractor trained
Epoch [4/20], Label Loss: 0.4976, Domain Loss: 1.0092
Source Label Accuracy: 77.73%
Target Label Accuracy: 89.66%
Source Domain Accuracy: 81.52%
Target Domain Accuracy: 43.65%

label trained
domain trained
feature extractor trained
Epoch [5/20], Lab

KeyboardInterrupt: 

In [42]:
def write_conf_results(lam, test_src, test_targ):
    file_name = 'results_conf_Adult_Sex_farcon_data_'+str(lam)+'.txt'
    my_str = 'Lambda: ' +str(lam) + '\n\n'
    for i in range(10):
        my_str += 'run: ' + str(i+1)+'/15\n'
        model_file = 'model_adult_'+str(lam)+'_'+ str(i) + '.pt'
        path = 'model_results/'+ model_file
        model = train_for_lambda(lam)
        torch.save(model.state_dict(), path)
        results = get_results(model, test_src, test_targ, lam)

        # test accuracy-- source, target, and overall
        my_str += 'Label Classifier Accuracy (Test Data) \n'
        my_str += 'Source (White): ' + str(results['source_label_accuracy']) + '\n'
        my_str += 'Target (Black): ' + str(results['target_label_accuracy'])+ '\n'
        my_str += 'Overall: ' + str(results['overall_label_accuracy'])+ '\n\n'
        # domain test accuracy-- source, target, and overall
        my_str += 'Domain Classifier Accuracy (Test Data) \n'
        my_str += 'Source (White): ' + str(results['source_domain_accuracy']) + '\n'
        my_str += 'Target (Black): ' + str(results['target_domain_accuracy'])+ '\n'
        my_str += 'Overall: ' + str(results['overall_domain_accuracy'])+ '\n\n'
        # false positives-- source, target, and overall
        my_str += 'False Positive Rate (Test Data) \n'
        my_str += 'Source (White): ' + str(results['fpr_white']) + '\n'
        my_str += 'Target (Black): ' + str(results['fpr_black'])+ '\n'
        my_str += '(Black FPR) / (White FPR): ' +  str(results['fpr_black']/results['fpr_white'])+ '\n'
        my_str += 'Overall: ' + str(results['fpr_overall'])+ '\n\n'
        my_str += 'Source Label Positive Rate: ' + str(results['source_label_positive_rate']) + '\n'
        my_str += 'Target Label Positive Rate: ' + str(results['target_label_positive_rate']) + '\n'
        my_str += 'Overall Label Positive Rate: ' + str(results['overall_label_positive_rate']) + '\n\n'

        print(my_str)

    print(my_str)
    with open(file_name, "w") as file:
        file.write(my_str)

In [43]:
for l in range(10):
    l = l * .1
    write_conf_results(l, source_test, target_test)

label trained
domain trained
feature extractor trained
Epoch [1/20], Label Loss: 0.7564, Domain Loss: 1.0351
Source Label Accuracy: 53.25%
Target Label Accuracy: 88.50%
Source Domain Accuracy: 100.00%
Target Domain Accuracy: 0.00%

label trained
domain trained
feature extractor trained
Epoch [2/20], Label Loss: 0.5504, Domain Loss: 1.0218
Source Label Accuracy: 75.92%
Target Label Accuracy: 90.92%
Source Domain Accuracy: 84.46%
Target Domain Accuracy: 34.60%

label trained
domain trained
feature extractor trained
Epoch [3/20], Label Loss: 0.4926, Domain Loss: 1.0382
Source Label Accuracy: 77.55%
Target Label Accuracy: 89.83%
Source Domain Accuracy: 81.11%
Target Domain Accuracy: 44.44%

label trained
domain trained
feature extractor trained
Epoch [4/20], Label Loss: 0.5153, Domain Loss: 1.0314
Source Label Accuracy: 77.55%
Target Label Accuracy: 89.66%
Source Domain Accuracy: 80.44%
Target Domain Accuracy: 48.25%

label trained
domain trained
feature extractor trained
Epoch [5/20], Lab

/var/folders/vs/r06tq8hs5p1_3hh8zgjvh93r0000gn/T/ipykernel_8266/1686696.py:26: RuntimeWarning: invalid value encountered in scalar divide
  my_str += '(Black FPR) / (White FPR): ' +  str(results['fpr_black']/results['fpr_white'])+ '\n'


Lambda: 0.30000000000000004

run: 1/15
Label Classifier Accuracy (Test Data) 
Source (White): 77.00622338683263
Target (Black): 88.77169105137801
Overall: 80.82918739635157

Domain Classifier Accuracy (Test Data) 
Source (White): 77.9233540779561
Target (Black): 43.38210275603947
Overall: 66.69983416252073

False Positive Rate (Test Data) 
Source (White): 26.129188926663428
Target (Black): 8.919646289888504
(Black FPR) / (White FPR): 0.3413671321724987
Overall: 19.467182616460782

Source Label Positive Rate: 0.4480838477611542
Target Label Positive Rate: 0.16059884428977966
Overall Label Positive Rate: 0.35467109084129333

run: 2/15
Label Classifier Accuracy (Test Data) 
Source (White): 77.46478873239437
Target (Black): 89.28206873086084
Overall: 81.3045881702598

Domain Classifier Accuracy (Test Data) 
Source (White): 78.30003275466754
Target (Black): 42.701599183395714
Overall: 66.73300165837479

False Positive Rate (Test Data) 
Source (White): 25.036425449247208
Target (Black): 8.26

/var/folders/vs/r06tq8hs5p1_3hh8zgjvh93r0000gn/T/ipykernel_8266/1686696.py:26: RuntimeWarning: invalid value encountered in scalar divide
  my_str += '(Black FPR) / (White FPR): ' +  str(results['fpr_black']/results['fpr_white'])+ '\n'


Lambda: 0.6000000000000001

run: 1/15
Label Classifier Accuracy (Test Data) 
Source (White): 67.44186046511628
Target (Black): 88.49948962232051
Overall: 74.28413488114981

Domain Classifier Accuracy (Test Data) 
Source (White): 100.0
Target (Black): 0.0
Overall: 67.5069098949696

False Positive Rate (Test Data) 
Source (White): 0.0
Target (Black): 0.0
(Black FPR) / (White FPR): nan
Overall: 0.0

Source Label Positive Rate: 0.0
Target Label Positive Rate: 0.0
Overall Label Positive Rate: 0.0


label trained
domain trained
feature extractor trained
Epoch [1/20], Label Loss: 0.8687, Domain Loss: 0.9974
Source Label Accuracy: 52.74%
Target Label Accuracy: 88.50%
Source Domain Accuracy: 100.00%
Target Domain Accuracy: 0.00%

label trained
domain trained
feature extractor trained
Epoch [2/20], Label Loss: 0.6944, Domain Loss: 0.9009
Source Label Accuracy: 53.55%
Target Label Accuracy: 88.50%
Source Domain Accuracy: 79.76%
Target Domain Accuracy: 47.09%

label trained
domain trained
feature 

...

Lambda: 0.8

run: 1/15
Label Classifier Accuracy (Test Data) 
Source (White): 78.11988208319686
Target (Black): 89.62232051718271
Overall: 81.85737976782752

Domain Classifier Accuracy (Test Data) 
Source (White): 74.95905666557485
Target (Black): 49.506634909833274
Overall: 66.68877833056938

False Positive Rate (Test Data) 
Source (White): 23.336571151044197
Target (Black): 7.8431372549019605
(Black FPR) / (White FPR): 0.3360878170206688
Overall: 17.338889715731508

Source Label Positive Rate: 0.4215525686740875
Target Label Positive Rate: 0.1500510424375534
Overall Label Positive Rate: 0.3333333432674408

run: 2/15
Label Classifier Accuracy (Test Data) 
Source (White): 78.16901408450704
Target (Black): 89.75842123171147
Overall: 81.93477059148701

Domain Classifier Accuracy (Test Data) 
Source (White): 73.40320995741894
Target (Black): 54.23613473970738
Overall: 67.17523493642896

False Positive Rate (Test Data) 
Source (White): 23.166585721223896
Target (Black): 7.650903498654364
(

/var/folders/vs/r06tq8hs5p1_3hh8zgjvh93r0000gn/T/ipykernel_8266/1686696.py:26: RuntimeWarning: invalid value encountered in scalar divide
  my_str += '(Black FPR) / (White FPR): ' +  str(results['fpr_black']/results['fpr_white'])+ '\n'


Lambda: 0.8

run: 1/15
Label Classifier Accuracy (Test Data) 
Source (White): 78.11988208319686
Target (Black): 89.62232051718271
Overall: 81.85737976782752

Domain Classifier Accuracy (Test Data) 
Source (White): 74.95905666557485
Target (Black): 49.506634909833274
Overall: 66.68877833056938

False Positive Rate (Test Data) 
Source (White): 23.336571151044197
Target (Black): 7.8431372549019605
(Black FPR) / (White FPR): 0.3360878170206688
Overall: 17.338889715731508

Source Label Positive Rate: 0.4215525686740875
Target Label Positive Rate: 0.1500510424375534
Overall Label Positive Rate: 0.3333333432674408

run: 2/15
Label Classifier Accuracy (Test Data) 
Source (White): 78.16901408450704
Target (Black): 89.75842123171147
Overall: 81.93477059148701

Domain Classifier Accuracy (Test Data) 
Source (White): 73.40320995741894
Target (Black): 54.23613473970738
Overall: 67.17523493642896

False Positive Rate (Test Data) 
Source (White): 23.166585721223896
Target (Black): 7.650903498654364
(

/var/folders/vs/r06tq8hs5p1_3hh8zgjvh93r0000gn/T/ipykernel_8266/1686696.py:26: RuntimeWarning: invalid value encountered in scalar divide
  my_str += '(Black FPR) / (White FPR): ' +  str(results['fpr_black']/results['fpr_white'])+ '\n'


Lambda: 0.9

run: 1/15
Label Classifier Accuracy (Test Data) 
Source (White): 78.34916475597772
Target (Black): 89.99659748213678
Overall: 82.13377556661139

Domain Classifier Accuracy (Test Data) 
Source (White): 70.50442188011792
Target (Black): 60.190541000340254
Overall: 67.15312327252626

False Positive Rate (Test Data) 
Source (White): 21.928120446818845
Target (Black): 7.22798923490965
(Black FPR) / (White FPR): 0.32962192324870365
Overall: 16.237535347521952

Source Label Positive Rate: 0.4048476815223694
Target Label Positive Rate: 0.14290575683116913
Overall Label Positive Rate: 0.319734662771225

run: 2/15
Label Classifier Accuracy (Test Data) 
Source (White): 78.10350474942679
Target (Black): 89.86049676760803
Overall: 81.92371475953566

Domain Classifier Accuracy (Test Data) 
Source (White): 73.3704552898788
Target (Black): 54.168084382443006
Overall: 67.13101160862355

False Positive Rate (Test Data) 
Source (White): 22.680913064594463
Target (Black): 7.535563244905806
(B

In [ ]:
# Get true positive and negative rates for test data !!!!!

In [22]:
true_pos_black = sum(y_test_target)
print('1s in y_test_target', true_pos_black)
true_pos_white = sum(y_test_source)
print('1s in y_test_source', true_pos_white)

length_white = len(y_test_source)
length_black = len(y_test_target)

true_neg_black = len(y_test_target) - true_pos_black
print('zeros in test black ', true_neg_black)
true_neg_white = len(y_test_source) - true_pos_white
print('zeros in test white ', true_neg_white)

1s in y_test_target tensor(358)
1s in y_test_source tensor(1965)
zeros in test black  tensor(2881)
zeros in test white  tensor(4565)


In [43]:
l = 2
model = train_for_lambda(l)
write_results(l, model, source_test, target_test)

label trained
domain trained
feature extractor trained
Epoch [1/20], Label Loss: 0.4687, Domain Loss: 0.5166
Source Label Accuracy: 73.77%
Target Label Accuracy: 88.30%
Source Domain Accuracy: 67.17%
Target Domain Accuracy: 93.86%

label trained
domain trained
feature extractor trained
Epoch [2/20], Label Loss: 0.4125, Domain Loss: 0.4655
Source Label Accuracy: 77.53%
Target Label Accuracy: 88.64%
Source Domain Accuracy: 49.05%
Target Domain Accuracy: 93.70%

label trained
domain trained
feature extractor trained
Epoch [3/20], Label Loss: 0.4908, Domain Loss: 0.4305
Source Label Accuracy: 78.00%
Target Label Accuracy: 89.13%
Source Domain Accuracy: 46.47%
Target Domain Accuracy: 90.00%

label trained
domain trained
feature extractor trained
Epoch [4/20], Label Loss: 0.5700, Domain Loss: 0.6319
Source Label Accuracy: 78.47%
Target Label Accuracy: 88.85%
Source Domain Accuracy: 67.44%
Target Domain Accuracy: 79.84%

label trained
domain trained
feature extractor trained
Epoch [5/20], Lab

In [44]:
l = 3
model = train_for_lambda(l)
write_results(l, model, source_test, target_test)

label trained
domain trained
feature extractor trained
Epoch [1/20], Label Loss: 0.5885, Domain Loss: 0.4238
Source Label Accuracy: 73.29%
Target Label Accuracy: 88.39%
Source Domain Accuracy: 64.96%
Target Domain Accuracy: 91.54%

label trained
domain trained
feature extractor trained
Epoch [2/20], Label Loss: 0.5158, Domain Loss: 0.3608
Source Label Accuracy: 76.86%
Target Label Accuracy: 88.55%
Source Domain Accuracy: 43.04%
Target Domain Accuracy: 93.49%

label trained
domain trained
feature extractor trained
Epoch [3/20], Label Loss: 0.4688, Domain Loss: 0.4645
Source Label Accuracy: 77.31%
Target Label Accuracy: 89.04%
Source Domain Accuracy: 41.83%
Target Domain Accuracy: 89.81%

label trained
domain trained
feature extractor trained
Epoch [4/20], Label Loss: 0.5515, Domain Loss: 0.7081
Source Label Accuracy: 77.98%
Target Label Accuracy: 89.23%
Source Domain Accuracy: 62.64%
Target Domain Accuracy: 83.30%

label trained
domain trained
feature extractor trained
Epoch [5/20], Lab

In [59]:
# do for larger lambda values
lams = [2, 3, 5, 10, 15, 100]
for l in lams:
    write_conf_results(l, source_test, target_test)


label trained
domain trained
feature extractor trained
Epoch [1/20], Label Loss: 0.7262, Domain Loss: 0.4768
Source Label Accuracy: 54.06%
Target Label Accuracy: 88.95%
Source Domain Accuracy: 67.68%
Target Domain Accuracy: 85.71%

label trained
domain trained
feature extractor trained
Epoch [2/20], Label Loss: 0.6750, Domain Loss: 0.4367
Source Label Accuracy: 53.88%
Target Label Accuracy: 88.95%
Source Domain Accuracy: 63.04%
Target Domain Accuracy: 90.89%

label trained
domain trained
feature extractor trained
Epoch [3/20], Label Loss: 0.5147, Domain Loss: 0.5232
Source Label Accuracy: 76.81%
Target Label Accuracy: 92.13%
Source Domain Accuracy: 64.90%
Target Domain Accuracy: 86.42%

label trained
domain trained
feature extractor trained
Epoch [4/20], Label Loss: 0.6077, Domain Loss: 0.6182
Source Label Accuracy: 77.75%
Target Label Accuracy: 92.10%
Source Domain Accuracy: 66.09%
Target Domain Accuracy: 84.90%

label trained
domain trained
feature extractor trained
Epoch [5/20], Lab

KeyboardInterrupt: 

In [60]:
def write_conf_results(lam, test_src, test_targ):
    file_name = 'results_conf_Adult_'+str(lam)+'.txt'
    my_str = 'Lambda: ' +str(lam) + '\n\n'
    for i in range(5):
        my_str += 'run: ' + str(i+1)+'/15\n'
        model = train_for_lambda(lam)
        results = get_results(model, test_src, test_targ, lam)
        # test accuracy-- source, target, and overall
        my_str += 'Label Classifier Accuracy (Test Data) \n'
        my_str += 'Source (White): ' + str(results['source_label_accuracy']) + '\n'
        my_str += 'Target (Black): ' + str(results['target_label_accuracy'])+ '\n'
        my_str += 'Overall: ' + str(results['overall_label_accuracy'])+ '\n\n'
        # domain test accuracy-- source, target, and overall
        my_str += 'Domain Classifier Accuracy (Test Data) \n'
        my_str += 'Source (White): ' + str(results['source_domain_accuracy']) + '\n'
        my_str += 'Target (Black): ' + str(results['target_domain_accuracy'])+ '\n'
        my_str += 'Overall: ' + str(results['overall_domain_accuracy'])+ '\n\n'
        # false positives-- source, target, and overall
        my_str += 'False Positive Rate (Test Data) \n'
        my_str += 'Source (White): ' + str(results['fpr_white']) + '\n'
        my_str += 'Target (Black): ' + str(results['fpr_black'])+ '\n'
        my_str += '(Black FPR) / (White FPR): ' +  str(results['fpr_black']/results['fpr_white'])+ '\n'
        my_str += 'Overall: ' + str(results['fpr_overall'])+ '\n\n'
        my_str += 'Source Label Positive Rate: ' + str(results['source_label_positive_rate']) + '\n'
        my_str += 'Target Label Positive Rate: ' + str(results['target_label_positive_rate']) + '\n'
        my_str += 'Overall Label Positive Rate: ' + str(results['overall_label_positive_rate']) + '\n\n'
        print(my_str)

    print(my_str)
    with open(file_name, "w") as file:
        file.write(my_str)

In [61]:
# do for larger lambda values
lams = [3, 5, 10, 15, 100]
for l in lams:
    write_conf_results(l, source_test, target_test)

label trained
domain trained
feature extractor trained
Epoch [1/20], Label Loss: 0.6827, Domain Loss: 0.7042
Source Label Accuracy: 54.20%
Target Label Accuracy: 88.95%
Source Domain Accuracy: 69.94%
Target Domain Accuracy: 80.06%

label trained
domain trained
feature extractor trained
Epoch [2/20], Label Loss: 0.7357, Domain Loss: 0.7250
Source Label Accuracy: 67.29%
Target Label Accuracy: 90.55%
Source Domain Accuracy: 67.42%
Target Domain Accuracy: 84.53%

label trained
domain trained
feature extractor trained
Epoch [3/20], Label Loss: 0.5820, Domain Loss: 0.7755
Source Label Accuracy: 77.65%
Target Label Accuracy: 92.03%
Source Domain Accuracy: 66.71%
Target Domain Accuracy: 84.32%

label trained
domain trained
feature extractor trained
Epoch [4/20], Label Loss: 0.5304, Domain Loss: 0.8294
Source Label Accuracy: 77.83%
Target Label Accuracy: 91.88%
Source Domain Accuracy: 66.47%
Target Domain Accuracy: 83.73%

label trained
domain trained
feature extractor trained
Epoch [5/20], Lab

In [62]:
for l in range(10):
    l = l * .1
    write_conf_results(l, source_test, target_test)

label trained
domain trained
feature extractor trained
Epoch [1/20], Label Loss: 0.8880, Domain Loss: 1.0431
Source Label Accuracy: 53.92%
Target Label Accuracy: 88.95%
Source Domain Accuracy: 72.54%
Target Domain Accuracy: 78.85%

label trained
domain trained
feature extractor trained
Epoch [2/20], Label Loss: 0.6180, Domain Loss: 0.9703
Source Label Accuracy: 70.92%
Target Label Accuracy: 91.48%
Source Domain Accuracy: 70.11%
Target Domain Accuracy: 85.06%

label trained
domain trained
feature extractor trained
Epoch [3/20], Label Loss: 0.5766, Domain Loss: 0.9508
Source Label Accuracy: 78.07%
Target Label Accuracy: 91.85%
Source Domain Accuracy: 68.59%
Target Domain Accuracy: 91.88%

label trained
domain trained
feature extractor trained
Epoch [4/20], Label Loss: 0.5118, Domain Loss: 0.9338
Source Label Accuracy: 78.52%
Target Label Accuracy: 91.63%
Source Domain Accuracy: 67.34%
Target Domain Accuracy: 95.59%

label trained
domain trained
feature extractor trained
Epoch [5/20], Lab